In [1]:
import numpy as np
import torch
from dqn.agent import Agent
from unityagents import UnityEnvironment
from collections import deque

In [2]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]

In [4]:
action_size = brain.vector_action_space_size
state_size = len(state)

In [5]:
agent = Agent(state_size=state_size, action_size=action_size)

In [6]:
def train(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, max_score=13.0):
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores_window.append(score)
        scores.append(score)
        eps = max(eps_end, eps*eps_decay)
        print(f"\rEpisode {i_episode}\tAverage Score: {np.mean(scores_window)}", end="")
        if i_episode % 100 == 0:
            print(f"\rEpisode {i_episode}\tAverage Score: {np.mean(scores_window)}")
        if np.mean(scores_window) >= max_score:
            print(f"\nEnvironment solved in {i_episode-100} episodes!\tAverage Score: {np.mean(scores_window)}")
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

In [7]:
try:
    scores = train()
except Exception as e:
    env.close()
    raise e

Episode 100	Average Score: -0.0680808080808081
Episode 200	Average Score: 0.151
Episode 300	Average Score: 0.61
Episode 400	Average Score: 0.87
Episode 500	Average Score: 0.61
Episode 600	Average Score: 0.22
Episode 700	Average Score: 0.53
Episode 800	Average Score: 0.49
Episode 900	Average Score: 0.46
Episode 1000	Average Score: 0.37
Episode 1100	Average Score: 0.51
Episode 1200	Average Score: 0.31
Episode 1233	Average Score: -0.02

KeyboardInterrupt: 

In [8]:
env.close()